## 1. How many Spam Emails are there?

In [42]:
def count_spam(path = "D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data", folders = ['enron1','enron2']):
    import os
    from pathlib import Path
    #path = Path("D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data")
    path = Path(path)
    spam_count = 0
    for folder in folders:
        file_names = os.listdir(path / folder / 'spam')
        for file_name in file_names:
            if 'spam' in file_name:
                spam_count += 1
    print('There are', spam_count, 'spam emails in', folders)

In [43]:
count_spam()

There are 2993 spam emails in ['enron1', 'enron2']


## 2. Structure the email data from the 2 directories into 1 dataframe with columns: Status, Subject, Body.

#### Specify the folders that will be included and the path leading to the folders.

In [44]:
folders = ['enron1','enron2']
status = ['spam','ham']
path1 = "D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\{folder}\\{status}"

#### Create the paths containing all the desired emails.

In [45]:
from itertools import product
directories = list(product(folders, status))
directories

[('enron1', 'spam'), ('enron1', 'ham'), ('enron2', 'spam'), ('enron2', 'ham')]

In [46]:
paths = []
for i in directories:
    paths.append(path1.format(folder=i[0],status=i[1]))
paths

['D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\spam',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\ham',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron2\\spam',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron2\\ham']

#### List all the files in each `path`.

In [47]:
import glob
file_paths = [glob.glob(pathname=path + '\\*') for path in paths]

#### Now we have all the file paths of the emails in the specified folders.

In [48]:
file_paths[0][0:5]

['D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\spam\\0006.2003-12-18.GP.spam.txt',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\spam\\0008.2003-12-18.GP.spam.txt',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\spam\\0017.2003-12-18.GP.spam.txt',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\spam\\0018.2003-12-18.GP.spam.txt',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\spam\\0026.2003-12-18.GP.spam.txt']

In [49]:
file_paths[1][0:5]

['D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\ham\\0001.1999-12-10.farmer.ham.txt',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\ham\\0002.1999-12-13.farmer.ham.txt',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\ham\\0003.1999-12-14.farmer.ham.txt',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\ham\\0004.1999-12-14.farmer.ham.txt',
 'D:\\Cadetship\\IntroDS\\HW2\\apper-eda\\Notebooks\\data\\enron-data\\enron1\\ham\\0005.1999-12-14.farmer.ham.txt']

#### Having acquired the file paths, we can now extract the status, subject and message body of the emails.

In [50]:
subjects = []
bodies = []
status = []

# for each path
for file_path in file_paths:
    # for each file in path
    for i in file_path:
        # for status
        if 'spam.txt' in i:
            status.append('spam')
        elif 'ham.txt' in i:
            status.append('ham')
        # extract the subject from email
        with open(i,'r',errors='ignore') as f:
            subject = f.readlines()[0]
            subject = subject[9:]
            subjects.append(subject)
        # extract body message from email
        with open(i,'r',errors='ignore') as f:
            body = f.readlines()
            body = body[1:]
            bodies.append(body)
            
# join each sub-elements of the bodies list
bodies = [' '.join(i) for i in bodies]

print('Status has', len(status), 'elements.')
print('Subject has', len(subjects), 'elements.')
print('Body has', len(bodies), 'elements.')

Status has 11026 elements.
Subject has 11026 elements.
Body has 11026 elements.


#### Now, use the lists to create a dataframe.

In [51]:
import pandas as pd

In [52]:
enron = pd.DataFrame(list(zip(status,subjects,bodies)), columns=['Status','Subject','Body'])
enron.sample(10)

,Status,Subject,Body
970,spam,collene medg stockplay of the day jeer\n,"watson ,\n medg - our best pick with rapid gro..."
7059,ham,re : vmi agreements\n,"ravi ,\n maybe stinson can get on this confere..."
2104,ham,enron methanol ; meter # : 988291\n,this is a follow up to the note i gave you on ...
2553,ham,re : new eol product\n,the texas gas group will definitely be rolling...
6885,ham,re : risk 2000 - boston\n,"oliver ,\n i apologize for the delay . i was t..."
8148,ham,message 4\n,"dear dr . kaminski ,\n this is a message of di..."
10921,ham,your advice is appreciated\n,"vince ,\n in the morning we were talking about..."
1450,spam,best free adult dating\n,search for sexual partners in your areaclick h...
1867,ham,cross training\n,in an attempt to continue cross training on ou...
1842,ham,per nelson ferries ( dealmaker ) : we do pay s...,please change the base purchase deals to refle...


## 3. Build a Naive Bayes classifier to classify whether emails are spam or not.

#### Import libraries

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [54]:
enron.sample(5)

,Status,Subject,Body
7442,ham,"rdi conference - may 17 - 19 , 2000\n","hi ,\n i ' d like to make a personal invitatio..."
5209,spam,complete online pharmacy with same day shipping\n,meridia ® is an fda - approved oral prescripti...
5924,spam,looking for a specific medication ? let us kno...,healthy living for everyday life .\n we rarely...
7686,ham,re : hello from london\n,"zimin ,\n i have a copy of the article .\n i a..."
4417,ham,oasis on - line\n,daren - - -\n oasis will be doing away with th...


In [55]:
enron.shape

(11026, 3)

#### Create `Spam` column from the `Status` column and delete `Status` column.

In [56]:
enron['Spam'] = [1 if i == 'spam' else 0 for i in enron['Status']]
enron.drop('Status', inplace=True, axis=1)

In [57]:
enron.sample(5)

,Subject,Body,Spam
3355,re : ami chokshi resigns\n,i have an exit interview scheduled for 2 pm on...,0
9349,chonawee supatgiat / corp / enron is out of th...,i will be out of the office from 11 / 27 / 200...,0
6225,new love tabs shop .\n,visit our llcensed online dragstore for the be...,1
7230,enron media / advertising\n,"simon ,\n please forward a copy of my ken rice...",0
10146,"re : "" expected tail loss "" for equity portfol...","everybody ,\n i attached here the equity portf...",0


#### Split data for training and testing.

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
train_X, test_X, train_y, test_y = train_test_split(enron["Body"], enron["Spam"], test_size=0.3, random_state=0)

#### Import libraries for text analysis

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB

In [61]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_train_X = vectorizer.fit_transform(train_X)

#### Train the model

In [62]:
classifier = GaussianNB()
classifier.fit(tfidf_train_X.toarray(), train_y)

GaussianNB()

## 4. What is the longest ham email?

#### Create a dataframe containing all ham emails.

In [63]:
hams = enron[enron['Spam'] == 0][['Body']]

#### Create a column containing the lengths of each emails.

In [64]:
hams['Lengths'] = [len(i) for i in hams['Body']]
hams.drop('Body',inplace=True,axis=1)

#### Get the index of the longest ham email.

In [65]:
longest_ham = hams.idxmax()
print(longest_ham)

Lengths    10765
dtype: int64


#### Print the longest ham email.

In [70]:
enron.iloc[longest_ham,1]

10765    fyi news articles from indian press .\n - - - ...
Name: Body, dtype: object

#### The longest ham email has `43888` characters.

In [71]:
hams.max()

Lengths    43888
dtype: int64

## 5. What is the accuracy of your model?

In [72]:
from sklearn.model_selection import cross_val_score

In [73]:
tfidf_test_X = vectorizer.transform(test_X)
print(tfidf_test_X.shape)
scores = cross_val_score(classifier, tfidf_test_X.toarray(), test_y, cv=5)
acc = scores.mean()
print("Accuracy: %0.2f percent" % (acc *100))

(3308, 58021)
Accuracy: 94.01 percent


**The model has an accuracy of `94.01%`.**

## 6. Include the Subject in the analysis of the emails, does the accuracy/performance of the model increase?

#### Create a new column containing the concatenation of `Subject` and `Body`.

In [74]:
enron['Total'] = [sub + bod for (sub, bod) in zip(enron.iloc[:,0],enron.iloc[:,1])]

In [75]:
enron.head(5)

,Subject,Body,Spam,Total
0,dobmeos with hgh my energy level has gone up !...,introducing\n doctor - formulated\n hgh\n huma...,1,dobmeos with hgh my energy level has gone up !...
1,your prescription is ready . . oxwq s f e\n,"low cost prescription medications\n soma , ult...",1,your prescription is ready . . oxwq s f e\nlow...
2,get that new car 8434\n,people nowthe weather or climate in any partic...,1,get that new car 8434\npeople nowthe weather o...
3,await your response\n,"dear partner ,\n we are a team of government o...",1,"await your response\ndear partner ,\n we are a..."
4,"coca cola , mbna america , nascar partner with...",stock\n profile\n about\n company\n investment...,1,"coca cola , mbna america , nascar partner with..."


In [76]:
train_X, test_X, train_y, test_y = train_test_split(enron["Total"], enron["Spam"], test_size=0.3, random_state=0)

In [77]:
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_train_X = vectorizer.fit_transform(train_X)

In [78]:
classifier = GaussianNB()
classifier.fit(tfidf_train_X.toarray(), train_y)

GaussianNB()

In [79]:
tfidf_test_X = vectorizer.transform(test_X)
print(tfidf_test_X.shape)
scores = cross_val_score(classifier, tfidf_test_X.toarray(), test_y, cv=5)
acc = scores.mean()
print("Accuracy: %0.2f percent" % (acc *100))

(3308, 58864)
Accuracy: 94.68 percent


**The accuracy improved by `0.67%`**

## Bonus : Answer questions 1-6 using Enron 1,2, and 3.

**Answer to bonus question is in the jupyter notebook `PS3 - Bonus - Jake.ipynb`.**